In [0]:
import torch
# Сверточный слой своими руками

In [0]:
# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


# Функция добавления нулей с четырех сторон каждого изображения
def get_padding2d(input_images):
    padded_images = torch.nn.functional.pad(input_images, (1,1, 1, 1), 'constant', 0).float();
    return padded_images

print(get_padding2d(input_images))


tensor([[[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  2.,  0.],
          [ 0.,  3.,  4.,  5.,  0.],
          [ 0.,  6.,  7.,  8.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  9., 10., 11.,  0.],
          [ 0., 12., 13., 14.,  0.],
          [ 0., 15., 16., 17.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 18., 19., 20.,  0.],
          [ 0., 21., 22., 23.,  0.],
          [ 0., 24., 25., 26.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 27., 28., 29.,  0.],
          [ 0., 30., 31., 32.,  0.],
          [ 0., 33., 34., 35.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 36., 37., 38.,  0.],
          [ 0., 39., 40., 41.,  0.],
          [ 0., 42., 43., 44.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 45., 46., 

In [0]:
import numpy as np

# напишите тут код, вычисляющий выходную размерность слоя свертки
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    # input_matrix_shape - входная размерность (число изображений в батче*число слоев в одном изображении*высота изображения*ширина изображения)
    # out_channels - количество каналов фильтров
    # kernel_size - размер фильтров
    # stride 
    # padding
    out_shape = np.array([]) 
    out_shape = np.append(out_shape, input_matrix_shape[0])
    out_shape = np.append(out_shape, out_channels)
    h = np.fix(((input_matrix_shape[2] + 2*padding - 1*(kernel_size - 1) - 1)/stride) + 1)
    w = np.fix(((input_matrix_shape[3] + 2*padding - 1*(kernel_size - 1) - 1)/stride) + 1)
    out_shape = np.append(out_shape, h)
    out_shape = np.append(out_shape, w)
    return out_shape

print(np.array_equal(
    calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0),
    [2, 10, 8, 8]))


True


In [0]:
# реализуем сверточный слой различными способами.
# Для удобства все наши реализации оформим в виде классов.
# Интерфейс классов сделаем одинаковым и максимально похожим на интерфейс стандартной реализации

# код для проверки:

In [0]:
import torch
from abc import ABC, abstractmethod

def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


# функция, создающая объект класса cls и возвращающая свертку от input_matrix
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)
 
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
              and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

True


In [0]:
# Если бы изображение было многослойным, например трехслойное - RGB, значит, 
# фильтры в ядре тоже должны быть трехслойные. 
# Каждый слой фильтра попиксельно умножается на соответствующий слой исходного изображения.
# То есть в данном случае после умножения получилось бы 4*3 = 12 произведений, 
# результаты которых складываются, и получается значение выходного пикселя.

# Если бы фильтров в ядре было больше одного, то добавился бы внешний цикл по фильтрам,
# внутри которого мы считаем свертку для каждого фильтра.

# Если бы во входном батче было более 1 изображения, то добавился бы еще один внешний 
# цикл по изображениям в батче.

In [0]:
# СВЕРТОЧНЫЙ СЛОЙ ЧЕРЕЗ ЦИКЛЫ.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        # Так у conv2D из pytorch:
        # conv2d = torch.nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size,
        #                               self.stride, padding=0, bias=False)
        # conv2d.weight.data = self.kernel
        # output_tensor = conv2d(input_tensor)

        # Так руками через циклы:
        # размеры выхода сверточного слоя
        batch_size, out_channels, output_height, output_width = calc_out_shape(
                                input_tensor.shape, 
                                self.out_channels,
                                self.kernel_size,
                                self.stride,
                                padding=0)
        # создадим выходной тензор, заполненный нулями         
        output_tensor = torch.zeros(batch_size, out_channels, output_height, output_width)
        # цикл по входным батчам(изображениям)
        for num_batch, batch in enumerate(input_tensor):
            # цикл по фильтрам (количество фильтров совпадает с количеством выходных каналов)  
            for num_kernel, kernel in enumerate(self.kernel):
               # цикл по размерам выходного изображения
                for i in range(output_height):
                    for j in range(output_width): 
                        # вырезаем кусочек из батча (сразу по всем входным каналам)
                        current_row = self.stride*i
                        current_column = self.stride*j
                        current_slice = batch[:, current_row:current_row + self.kernel_size,
                                              current_column:current_column + self.kernel_size]
                        # умножаем кусочек на фильтр
                        res = float((current_slice * kernel).sum())
                        # заполняем ячейку в выходном тензоре
                        output_tensor[num_batch,num_kernel,i,j] = res
        return output_tensor

# Корректность реализации определится в сравнии со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dLoop))

True


In [0]:
# СВЕРТОЧНЫЙ СЛОЙ ЧЕРЕЗ МАТРИЧНОЕ УМНОЖЕНИЕ.
class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        _, in_channels, in_height, in_width = torch_input.shape
        ku_size = [self.out_channels, output_height, output_width, in_channels, in_height, in_width]
        kernel_unsqueezed = torch.zeros(ku_size, dtype=torch.float32)
        for i in range(output_height):
            for j in range(output_width):
                h_slice = slice(i*self.stride, i*self.stride+self.kernel_size)
                w_slice = slice(j*self.stride, j*self.stride+self.kernel_size)
                kernel_unsqueezed[:, i, j, :, h_slice, w_slice] = self.kernel.type(torch.float32)
        return kernel_unsqueezed.view(-1, in_channels*in_height*in_width)

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrix))

True


In [0]:
# СВЕРТОЧНЫЙ СЛОЙ ЧЕРЕЗ МАТРИЧНОЕ УМНОЖЕНИЕ. УЛУЧШЕННЫЙ
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        c_out, c_in, kernel_size, _ = self.kernel.shape
        converted_kernel = self.kernel.view(c_out, -1)
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        n, c_in, h_in, w_in = torch_input.shape
        c_out, _, kernel_size, _ = self.kernel.shape
        converted_input = torch.zeros(n, c_in * kernel_size * kernel_size, h_out * w_out)
        for i in range(n):
            current_image = torch_input[i]
            for p in range(h_out):
                for q in range(w_out):
                    slice_ = current_image[:, 
                                           p*self.stride:p*self.stride+kernel_size, 
                                           q*self.stride:q*self.stride+kernel_size
                    ]

                    converted_input[i][:, p * h_out + q] = slice_.flatten()        
        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(0, 1).view(torch_input.shape[0],
                                                     self.out_channels, output_height,
                                                     output_width).transpose(1, 3).transpose(2, 3)